In [2]:
import requests as re
import pandas as pd
import numpy as np
import os
from bs4 import BeautifulSoup

In [3]:
pd.set_option('display.max_colwidth', -1)

In [4]:
%%time
def getForums(site_url):
    col=['forum','subforum','forum_url']
    top_df = pd.DataFrame(columns=col)
    r = re.get(site_url)
    page = r.text
    soup = BeautifulSoup(page, 'html.parser')
    for element in soup.find_all('td', {"class": "alt1Active"}):
        forum_title_div = element.div
        forum_link = forum_title_div.a
        forum_url = site_url + forum_link['href']
        if('.com' in forum_link['href']):
            forum_url = forum_link['href']
        forum_title = forum_link.strong.string
        subforum = ''

        row = pd.DataFrame([[forum_title,subforum,forum_url]], columns=col)
        if(len(top_df)==0):
            top_df = row
        else:
            top_df = top_df.append(row, ignore_index=True) #df.append doesn't work inplace

        for child in element.find_all('div', {"style" : "margin-top:6px"}):
            if child.find('strong') != None: #has a Sub-Forums section
                for subforum_link in child.find_all('a'):
                    subforum = subforum_link.contents[0]
                    forum_url = site_url + subforum_link['href']
                    if('.com' in subforum_link['href']):
                        forum_url = subforum_link['href']
                    row = pd.DataFrame([[forum_title,subforum,forum_url]], columns=col)
                    if(len(top_df)==0):
                        top_df = row
                    else:
                        top_df = top_df.append(row, ignore_index=True) #df.append doesn't work inplace
    return top_df

Wall time: 0 ns


In [20]:
url = 'https://forums.hardwarezone.com.sg'
hwz = getForums(url)
print(len(hwz))
hwz

119


,forum,subforum,forum_url
0,Tech Show Central (IT Show 2018),,https://forums.hardwarezone.com.sg/tech-show-central-show-2018-394/
1,Hardware Clinic,,https://forums.hardwarezone.com.sg/hardware-clinic-2/
2,Hardware Clinic,CORBELL eCustomer Service Center,https://forums.hardwarezone.com.sg/corbell-ecustomer-service-center-166/
3,Hardware Clinic,CONVERGENT eCustomer Service Center,https://forums.hardwarezone.com.sg/convergent-ecustomer-service-center-184/
4,Hardware Clinic,ASUS eCustomer Service Center,https://forums.hardwarezone.com.sg/asus-ecustomer-service-center-336/
5,Hardware Clinic,Avertek eCustomer Service Center,https://forums.hardwarezone.com.sg/avertek-ecustomer-service-center-388/
6,Hardware Clinic,Hardware Clinic Knowledge Base,https://forums.hardwarezone.com.sg/hardware-clinic-knowledge-base-33/
7,Hardware Clinic,Personal Projects and Reviews Den,https://forums.hardwarezone.com.sg/personal-projects-reviews-den-72/
8,Notebook Clinic,,https://forums.hardwarezone.com.sg/notebook-clinic-77/
9,Notebook Clinic,Gaming / Entertainment Class Notebook Hub,https://forums.hardwarezone.com.sg/gaming-entertainment-class-notebook-hub-293/


Build a table of forums and subforums

In [6]:
%%time
def getThreads(forum_url):
    print(forum_url)
    site_url = 'https://forums.hardwarezone.com.sg'
    lastForumPage = False
    forum_cols = ['forum_url', 'thread', 'thread_url']
    forum_df = pd.DataFrame(columns=forum_cols)
    forum_page_url = forum_url
    while(not lastForumPage):
        r2 = re.get(forum_page_url)
        forum_page = r2.text
        forum_page_soup = BeautifulSoup(forum_page, 'html.parser')

        if (forum_page_soup.find('a', text='Next ›') == None):
            lastForumPage = True
        else:
            forum_page_url = site_url + forum_page_soup.find('a', text='Next ›')['href']

        for element in forum_page_soup.find_all('a', {"id": lambda x: x and 'thread_title_' in x}):
            row = pd.DataFrame([[forum_url, element.contents[0], site_url + element['href']]], columns=forum_cols)
            if(len(forum_df)==0):
                forum_df = row
            else:
                forum_df = forum_df.append(row, ignore_index=True) #df.append doesn't work inplace
    return forum_df

Wall time: 0 ns


In [7]:
parenting = getThreads('https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/')
print(len(parenting))
parenting

https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/
154


,forum_url,thread,thread_url
0,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/,"Welcome to HWZ's Parenting, Kids & Early Learning forum",https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/welcome-hwzs-parenting-kids-early-learning-forum-5684416.html
1,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/,Milk powder,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/milk-powder-5773651.html
2,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/,I suspect my child has ADHD,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/i-suspect-my-child-has-adhd-5816748.html
3,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/,Do u lose your temper when teaching your child?,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/do-u-lose-your-temper-when-teaching-your-child-5785587.html
4,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/,Cry when feeding,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/cry-when-feeding-5723877.html
5,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/,"Newborn, nanny, infant care or maid after maternity leave?",https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/newborn-nanny-infant-care-maid-after-maternity-leave-5805042.html
6,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/,Anyone went thru abortion before?,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/anyone-went-thru-abortion-before-5779121.html
7,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/,Selling Obscene Films and Used Panties Can Be Illegal In Singapore,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/selling-obscene-films-used-panties-can-illegal-singapore-5815478.html
8,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/,Potty training,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/potty-training-5813079.html
9,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/,Recommendations for Gynae,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/recommendations-gynae-5784603.html


Get threads in a forum/sub-forum - all pages

In [8]:
%%time
def getPosts(thread_url):
    print(thread_url)
    site_url = 'https://forums.hardwarezone.com.sg'
    lastThreadPage = False
    thread_cols = ['thread_url', 'userid', 'timestamp', 'post_text'] #, 'post_number', 'post_order', 'first_quote_post_number', 'likes_userid'
    thread_df = pd.DataFrame(columns=thread_cols)
    thread_page_url = thread_url

    while(not lastThreadPage):
        #print(thread_page_url)
        r3 = re.get(thread_page_url)
        thread_page = r3.text
        thread_page_soup = BeautifulSoup(thread_page, 'html.parser')

        if (thread_page_soup.find('a', text='Next ›') == None):
            lastThreadPage = True
        else:
            thread_page_url = site_url + thread_page_soup.find('a', text='Next ›')['href']

        thread_page_posts = thread_page_soup.find('div', {'id': 'posts'})
        
        try: 
            for post in thread_page_posts.find_all('div', {'class': 'post-wrapper'}):
                userid_url = post.find('a', {'class': 'bigusername'})['href']
                userid = ''.join(filter(lambda x: x.isdigit(), userid_url))

                datetime_raw = post.find('a', {'name': lambda x: x and x.find('post') == 0}).nextSibling.strip()
                date_list = datetime_raw.split(',')[0].split('-')
                iso_date = '-'.join(list(reversed(date_list)))
                hour = int(datetime_raw.split(' ')[1][0:2])
                if(datetime_raw.split(' ')[2] == 'PM' and hour < 12):
                    hour += 12
                hour_str = str(hour)
                if(hour < 10):
                    hour_str = '0' + str(hour)
                minute = datetime_raw.split(':')[1][0:2]
                iso_datetime = iso_date + 'T' + hour_str + ':' + minute

                post_text = post.find('div', {'class': 'post_message'}).get_text(' ', strip=True)

                row = pd.DataFrame([[thread_url, userid, iso_datetime, post_text]], columns=thread_cols)
                if(len(thread_df)==0):
                    thread_df = row
                else:
                    thread_df = thread_df.append(row, ignore_index=True) #df.append doesn't work inplace
        except:
            row = pd.DataFrame([[thread_url, "", "", ""]], columns=thread_cols) #posts missing, thread may have been deleted
            if(len(thread_df)==0):
                thread_df = row
            else:
                thread_df = thread_df.append(row, ignore_index=True) #df.append doesn't work inplace
    return thread_df

Wall time: 0 ns


In [9]:
pointofchildren = getPosts('https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/actually-what-point-children-5604908.html')
print(len(pointofchildren))
pointofchildren

https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/actually-what-point-children-5604908.html
232


,thread_url,userid,timestamp,post_text
0,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/actually-what-point-children-5604908.html,642993,2017-04-21T10:09,"Most of the reasons for babies in SG is.......: -Because it is norm/Appease in laws -Accident/Abortion is EVIL -Insurance for old muahahaha -Love children much -Need a substitute doll to fulfill my dead dreams What else are there? other than point 4, really no point to keep wor. Unlike last time generation peepur..now cheaper to manage wealth than to rear babies since we are more educated ."
1,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/actually-what-point-children-5604908.html,322359,2017-04-21T10:44,Can't finish all the money from dad
2,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/actually-what-point-children-5604908.html,370175,2017-04-21T11:55,"In Singapore? No point. Unless you are filthy rich, there are utterly no justify-able reasons to have kids."
3,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/actually-what-point-children-5604908.html,306175,2017-04-21T11:58,Ask your parents why bring you up
4,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/actually-what-point-children-5604908.html,270705,2017-04-21T12:00,what is the point of ur existence...no point right? go kill urself
5,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/actually-what-point-children-5604908.html,103529,2017-04-21T12:09,Tiagong government is bigger than your parents.
6,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/actually-what-point-children-5604908.html,40851,2017-04-21T12:11,1) To create jobs by having more schools = teachers = janitor = etc 2) To make PAP happy
7,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/actually-what-point-children-5604908.html,327041,2017-04-21T12:11,if your parents think the same way u won't even be here typing this nonsense
8,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/actually-what-point-children-5604908.html,558576,2017-04-21T12:14,Point 4 still not enuff?
9,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/actually-what-point-children-5604908.html,237992,2017-04-21T12:14,"In modern day context, anything but love children lol"


In [10]:
hwz.loc[hwz['forum_url'] == 'https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/']

,forum,subforum,forum_url
70,"Parenting, Kids & Early Learning",,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/


In [11]:
test = pd.merge(hwz, parenting, left_on = 'forum_url', right_on = 'forum_url', how = 'inner')
test

,forum,subforum,forum_url,thread,thread_url
0,"Parenting, Kids & Early Learning",,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/,"Welcome to HWZ's Parenting, Kids & Early Learning forum",https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/welcome-hwzs-parenting-kids-early-learning-forum-5684416.html
1,"Parenting, Kids & Early Learning",,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/,Milk powder,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/milk-powder-5773651.html
2,"Parenting, Kids & Early Learning",,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/,I suspect my child has ADHD,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/i-suspect-my-child-has-adhd-5816748.html
3,"Parenting, Kids & Early Learning",,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/,Do u lose your temper when teaching your child?,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/do-u-lose-your-temper-when-teaching-your-child-5785587.html
4,"Parenting, Kids & Early Learning",,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/,Cry when feeding,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/cry-when-feeding-5723877.html
5,"Parenting, Kids & Early Learning",,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/,"Newborn, nanny, infant care or maid after maternity leave?",https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/newborn-nanny-infant-care-maid-after-maternity-leave-5805042.html
6,"Parenting, Kids & Early Learning",,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/,Anyone went thru abortion before?,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/anyone-went-thru-abortion-before-5779121.html
7,"Parenting, Kids & Early Learning",,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/,Selling Obscene Films and Used Panties Can Be Illegal In Singapore,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/selling-obscene-films-used-panties-can-illegal-singapore-5815478.html
8,"Parenting, Kids & Early Learning",,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/,Potty training,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/potty-training-5813079.html
9,"Parenting, Kids & Early Learning",,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/,Recommendations for Gynae,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/recommendations-gynae-5784603.html


In [12]:
%%time
# hwz['forum_url'].apply(lambda x: pd.merge(hwz, getThreads(x), left_on = 'forum_url', right_on = 'forum_url', how = 'inner')
allThreads = pd.concat([getThreads(forum_url) for forum_url in hwz['forum_url']])

https://forums.hardwarezone.com.sg/tech-show-central-show-2018-394/
https://forums.hardwarezone.com.sg/hardware-clinic-2/
https://forums.hardwarezone.com.sg/corbell-ecustomer-service-center-166/
https://forums.hardwarezone.com.sg/convergent-ecustomer-service-center-184/
https://forums.hardwarezone.com.sg/asus-ecustomer-service-center-336/
https://forums.hardwarezone.com.sg/avertek-ecustomer-service-center-388/
https://forums.hardwarezone.com.sg/hardware-clinic-knowledge-base-33/
https://forums.hardwarezone.com.sg/personal-projects-reviews-den-72/
https://forums.hardwarezone.com.sg/notebook-clinic-77/
https://forums.hardwarezone.com.sg/gaming-entertainment-class-notebook-hub-293/
https://forums.hardwarezone.com.sg/tablet-den-328/
https://forums.hardwarezone.com.sg/notebook-clinic-knowledge-base-339/
https://forums.hardwarezone.com.sg/internet-bandwidth-networking-clinic-4/
https://forums.hardwarezone.com.sg/next-generation-broadband-network-ngbn-forum-320/
https://forums.hardwarezone.co

Forever and a day just to get the thread urls

Next time I should write these so that they write to file and pick up where they left off maybe

But threads get updated in real time, so maybe this has to be done at one go anyway.

In [13]:
allThreads_csv = os.path.join("data", "allthreads-2018-04-24T1300.csv")

In [14]:
allThreads.to_csv(allThreads_csv, encoding='utf-8', index=False)

Better write that to file

In [15]:
allThreads_df = pd.read_csv(allThreads_csv)
len(allThreads_df)

156448

Now to get all the posts from all those threads

In [16]:
allThreads_df.loc[allThreads_df['thread_url'] == 'https://forums.hardwarezone.com.sg/showthread.php?t=5580281']

,forum_url,thread,thread_url
630,https://forums.hardwarezone.com.sg/tech-show-central-show-2018-394/,Credit card instalment plans during IT show,https://forums.hardwarezone.com.sg/showthread.php?t=5580281


In [17]:
%%time
allPosts = pd.concat([getPosts(thread_url) for thread_url in allThreads_df['thread_url']])

https://forums.hardwarezone.com.sg/tech-show-central-show-2018-394/%2A%2A-show-2018-show-preview-tech-guides-past-show-brochures%2A%2A-5788288.html
https://forums.hardwarezone.com.sg/tech-show-central-show-2018-394/%2A%2A-show-2018-tech-show-portal-launched-floor-plans-brochures-more-%2A%2A-5788382.html
https://forums.hardwarezone.com.sg/tech-show-central-show-2018-394/guide-how-buy-ideal-gaming-machine-5764374.html
https://forums.hardwarezone.com.sg/tech-show-central-show-2018-394/%5Bguide%5D-printers-some-things-you-might-want-take-note-5366804.html
https://forums.hardwarezone.com.sg/tech-show-central-show-2018-394/tech-show-central-guidelines-must-read-5622593.html
https://forums.hardwarezone.com.sg/tech-show-central-show-2018-394/mesh-wifi-5682793.html
https://forums.hardwarezone.com.sg/tech-show-central-show-2018-394/recommendation-true-wireless-earbuds-5793321.html
https://forums.hardwarezone.com.sg/tech-show-central-show-2018-394/m1-fibre-deals-5729826.html
https://forums.hardwa

https://forums.hardwarezone.com.sg/tech-show-central-show-2018-394/any-good-deals-ip-cams-webcams-5793273.html
https://forums.hardwarezone.com.sg/tech-show-central-show-2018-394/networking-show-too-cheap-5792838.html
https://forums.hardwarezone.com.sg/tech-show-central-show-2018-394/any-laptop-14-intro-5791767.html
https://forums.hardwarezone.com.sg/tech-show-central-show-2018-394/smart-speaker-recommendation-5793232.html
https://forums.hardwarezone.com.sg/tech-show-central-show-2018-394/any-router-trade-tp-link-5792168.html
https://forums.hardwarezone.com.sg/tech-show-central-show-2018-394/new-coffee-lake-cpu-desktop-5791553.html
https://forums.hardwarezone.com.sg/tech-show-central-show-2018-394/asus-laptop-5791210.html
https://forums.hardwarezone.com.sg/tech-show-central-show-2018-394/asus-zenbook-flip-s-ux370ua-hp-spectre-360-a-5792830.html
https://forums.hardwarezone.com.sg/tech-show-central-show-2018-394/beware-counterfeit-laptop-notebook-found-expo-2017-a-5593610.html
https://for

https://forums.hardwarezone.com.sg/tech-show-central-show-2018-394/anyone-know-good-place-buy-thoese-wireless-controllable-bulb-5750123.html
https://forums.hardwarezone.com.sg/tech-show-central-show-2018-394/gaming-speaker-5780281.html
https://forums.hardwarezone.com.sg/tech-show-central-show-2018-394/any-good-android-tv-box-offer-5578541.html
https://forums.hardwarezone.com.sg/tech-show-central-show-2018-394/trade-recommend-business-laptop-windows-tablet-5752617.html
https://forums.hardwarezone.com.sg/tech-show-central-show-2018-394/actually-show-got-any-true-good-deals-ornot-ah-5585275.html
https://forums.hardwarezone.com.sg/tech-show-central-show-2018-394/new-surface-pro-5681248.html
https://forums.hardwarezone.com.sg/tech-show-central-show-2018-394/%5Btechshow-central%5D-happy-lunar-new-year-2018-a-5776926.html
https://forums.hardwarezone.com.sg/tech-show-central-show-2018-394/lenovo-aftershock-pc-5766831.html
https://forums.hardwarezone.com.sg/tech-show-central-show-2018-394/anywh

https://forums.hardwarezone.com.sg/tech-show-central-show-2018-394/enquiry-sitex-participant-5728569.html
https://forums.hardwarezone.com.sg/tech-show-central-show-2018-394/teleco-deals-5728146.html
https://forums.hardwarezone.com.sg/tech-show-central-show-2018-394/cef-show-nothing-much-5719133.html
https://forums.hardwarezone.com.sg/tech-show-central-show-2018-394/canon-digital-camera-sale-5717235.html
https://forums.hardwarezone.com.sg/home-theatre-audiophiles-13/lg-sh7-sound-bar-not-able-play-songs-purchased-itune-songs-5720385.html
https://forums.hardwarezone.com.sg/tech-show-central-show-2018-394/lenovos-star-wars-jedi-challenges-smartphone-powered-ar-experience-did-you-try-5719832.html
https://forums.hardwarezone.com.sg/tech-show-central-show-2018-394/dont-get-cheated-3pm-sg-scam-5718853.html
https://forums.hardwarezone.com.sg/tech-show-central-show-2018-394/any-portable-chargers-there-5718942.html
https://forums.hardwarezone.com.sg/tech-show-central-show-2018-394/eubiq-pricing-5

https://forums.hardwarezone.com.sg/tech-show-central-show-2018-394/apple-trade-prices-5683328.html
https://forums.hardwarezone.com.sg/tech-show-central-show-2018-394/sim-card-router-lan-ports-5683183.html
https://forums.hardwarezone.com.sg/tech-show-central-show-2018-394/surface-pro-4-promo-5628452.html
https://forums.hardwarezone.com.sg/tech-show-central-show-2018-394/%5Baudio%5D-sony-headphones-sale-5682490.html
https://forums.hardwarezone.com.sg/tech-show-central-show-2018-394/why-singtel-booth-right-back-while-m1-front-5681100.html
https://forums.hardwarezone.com.sg/tech-show-central-show-2018-394/any-intel-stick-pc-5683234.html
https://forums.hardwarezone.com.sg/tech-show-central-show-2018-394/dell-monitor-booth-5682954.html
https://forums.hardwarezone.com.sg/tech-show-central-show-2018-394/samsung-c32hg70-31-5-c34f791-34-a-5682577.html
https://forums.hardwarezone.com.sg/tech-show-central-show-2018-394/msi-booth-can-buy-5682676.html
https://forums.hardwarezone.com.sg/tech-show-cen

...

https://forums.hardwarezone.com.sg/console-gaming-383/arcade360-funbox-classical-650-games-plug-play-5478384.html
https://forums.hardwarezone.com.sg/console-gaming-383/estimated-delivery-date-xbox-one-5492941.html
https://forums.hardwarezone.com.sg/console-gaming-383/ps4-spoilt-5483715.html
https://forums.hardwarezone.com.sg/console-gaming-383/help-i-cant-buy-anything-psn-asia-5477619.html
https://forums.hardwarezone.com.sg/console-gaming-383/grand-there-auto-5-a-5452341.html
https://forums.hardwarezone.com.sg/console-gaming-383/monster-hunter-direct-27-10-16-7pm-5489340.html
https://forums.hardwarezone.com.sg/console-gaming-383/sfv-collectors-edition-5491308.html
https://forums.hardwarezone.com.sg/console-gaming-383/ps4-based-clan-terminus-now-opened-destiny-5491352.html
https://forums.hardwarezone.com.sg/console-gaming-383/trade-x360-5483883.html
https://forums.hardwarezone.com.sg/console-gaming-383/cross-save-issues-5479360.html
https://forums.hardwarezone.com.sg/console-gaming-383/

https://forums.hardwarezone.com.sg/console-gaming-383/sony-rumoured-upgrade-ps4-faster-cpu-gpu-ram-5350206.html
https://forums.hardwarezone.com.sg/console-gaming-383/hori-pad-fps-plus-playstation-4-a-5349760.html
https://forums.hardwarezone.com.sg/console-gaming-383/any-ps4-game-recommendation-recently-5347963.html
https://forums.hardwarezone.com.sg/console-gaming-383/does-anyone-has-psn-troubles-5339135.html
https://forums.hardwarezone.com.sg/console-gaming-383/xbox-one-elite-controller-5320685.html
https://forums.hardwarezone.com.sg/console-gaming-383/ps4-wireless-stereo-headset-problem-5346440.html
https://forums.hardwarezone.com.sg/console-gaming-383/game-watch-nintendo-5342180.html
https://forums.hardwarezone.com.sg/console-gaming-383/gears-war-4-a-5343811.html
https://forums.hardwarezone.com.sg/console-gaming-383/whats-your-favourite-moment-sq-uncovered-ffxv-event-5336090.html
https://forums.hardwarezone.com.sg/console-gaming-383/ffxv-ultimate-collectors-ed-anyone-anywhere-533912

https://forums.hardwarezone.com.sg/handheld-mobile-gaming-384/%5Bandroid-ios%5D-official-line-lets-get-rich-thread-part-2-a-5101775.html
https://forums.hardwarezone.com.sg/handheld-mobile-gaming-384/%5Bandroid-ios%5D-bleach-brave-souls-english-version-released-5282196.html
https://forums.hardwarezone.com.sg/handheld-mobile-gaming-384/%5Bios-android%5D-wwe-champions-5603417.html
https://forums.hardwarezone.com.sg/handheld-mobile-gaming-384/%5Bios-android%5D-seven-knights-%5Benglish-asia%5D-5207359.html
https://forums.hardwarezone.com.sg/handheld-mobile-gaming-384/hero-hunters-5768853.html
https://forums.hardwarezone.com.sg/handheld-mobile-gaming-384/maplestory-m-5767326.html
https://forums.hardwarezone.com.sg/handheld-mobile-gaming-384/clash-royale-new-clan-5764577.html
https://forums.hardwarezone.com.sg/handheld-mobile-gaming-384/pokemon-go-mystic-air-force-5763124.html
https://forums.hardwarezone.com.sg/handheld-mobile-gaming-384/%5Bandroid-ios%5D-digimon-linkz-currently-japanese-5333

https://forums.hardwarezone.com.sg/handheld-mobile-gaming-384/%5Bofficial%5D-final-fantasy-brave-exvius-5407905.html
https://forums.hardwarezone.com.sg/handheld-mobile-gaming-384/%5Bios-android%5D-war-crown-5610758.html
https://forums.hardwarezone.com.sg/handheld-mobile-gaming-384/modern-combat-5-a-5661090.html
https://forums.hardwarezone.com.sg/handheld-mobile-gaming-384/%5Bandroid-ios%5D-torchlight-legend-continues-5660371.html
https://forums.hardwarezone.com.sg/handheld-mobile-gaming-384/%5Bios-android%5D-dragon-blaze-5013531.html
https://forums.hardwarezone.com.sg/handheld-mobile-gaming-384/%5Bandroid-ios%5D-cosmos-guardians-action-rpg-strategic-sci-fi-5641594.html
https://forums.hardwarezone.com.sg/handheld-mobile-gaming-384/idolm%40ster-million-live-theater-days-5657358.html
https://forums.hardwarezone.com.sg/handheld-mobile-gaming-384/transformers-forged-fight-5598119.html
https://forums.hardwarezone.com.sg/handheld-mobile-gaming-384/%5Bios-android%5D-danmemo-5642273.html
https:

SSLError: HTTPSConnectionPool(host='forums.hardwarezone.com.sg', port=443): Max retries exceeded with url: /handheld-mobile-gaming-384/ios-android-fantasy-war-tactics-5226748-92.html (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')",),))

In [18]:
allPosts_csv = os.path.join('data', "allposts-2018-04-24T1300.csv")

In [19]:
allPosts.to_csv(allPosts_csv, encoding='utf-8', index=False)

NameError: name 'allPosts' is not defined

In [ ]:
allPosts_df = pd.read_csv(allPosts_csv)
len(allPosts_df)

In [ ]:
%%time
forumPosts = pd.merge(allThreads_df, allPosts_df, left_on = "thread_url", right_on = "thread_url", how = "inner")

In [ ]:
%%time
sitePosts = pd.merge(hwz, forumPosts, left_on = "forum_url", right_on = "forum_url", how = "inner")

In [ ]:
sitePosts_csv = os.join("data", "siteposts-2018-04-24T1300.csv")

In [ ]:
sitePosts.to_csv(sitePosts_csv, encoding='utf-8', index=False)

In [ ]:
sitePosts_df = pd.read_csv(sitePosts_csv)
len(sitePosts_df)